In [ ]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier


In [13]:
df5=pd.read_csv('/content/features')

In [14]:
df6=pd.read_csv('/content/target')

In [15]:
bins = pd.cut(df6['length of service in months'], bins=31)

# assign a category label to each value based on the bin it falls into
categories = pd.factorize(bins, sort=True)[0]

# add the new categorical column to the dataframe
df6['length_of_service_categories'] = categories

In [16]:
# print the ranges of the bins and the number of values in each bin
print(bins.describe())

# print the number of values in each category
print(bins.value_counts())

count             17573
unique               23
top       (0.628, 13.0]
freq              11404
Name: length of service in months, dtype: object
(0.628, 13.0]     11404
(13.0, 25.0]       2902
(25.0, 37.0]       1547
(37.0, 49.0]        835
(49.0, 61.0]        582
(61.0, 73.0]        206
(73.0, 85.0]         22
(85.0, 97.0]         16
(97.0, 109.0]        14
(109.0, 121.0]        8
(145.0, 157.0]        7
(121.0, 133.0]        6
(133.0, 145.0]        4
(241.0, 253.0]        3
(181.0, 193.0]        3
(169.0, 181.0]        3
(193.0, 205.0]        2
(205.0, 217.0]        2
(157.0, 169.0]        2
(325.0, 337.0]        2
(217.0, 229.0]        1
(313.0, 325.0]        1
(361.0, 373.0]        1
(229.0, 241.0]        0
(253.0, 265.0]        0
(265.0, 277.0]        0
(277.0, 289.0]        0
(289.0, 301.0]        0
(301.0, 313.0]        0
(337.0, 349.0]        0
(349.0, 361.0]        0
Name: length of service in months, dtype: int64


In [17]:
df6=df6.drop('length of service in months',axis=1)

In [ ]:
df7 = pd.concat([df5, df6], axis=1)
df7.head

,Current Employee [1=YES / 0=No],Contracted hours,Job Title,Certification,Other,Accounts Skills,Admin Skills,FlexForce,General Skills,Health and Social Care,...,Total cars or vans,Residents in households,Schoolchildren or full-time students aged 4+ at their non term-time address,Married,In a registered same-sex civil partnership,Separated (but still legally married or in a same-sex civil partnership),Widowed (or surviving partner from a same-sex civil partnership),"Candidate previous employee [1 = yes, 0 = no]",salary,length_of_service_categories
0,0,22.0,0,0,0,0,0,0,0,0,...,181,304,4,136,0,6,35,0,9.55,2
1,0,44.0,0,0,0,0,0,0,0,0,...,174,232,6,82,0,8,7,1,8.72,1
2,1,33.0,0,0,0,0,0,0,0,0,...,26,117,0,16,0,1,0,0,10.50,0
3,0,38.5,0,0,0,0,0,0,0,0,...,45,273,0,55,0,10,17,0,9.55,3
4,0,22.0,0,0,0,0,0,0,0,0,...,120,310,2,122,0,2,11,0,9.55,12


In [ ]:
df=pd.read_csv('/content/data1.csv',index_col=0)

In [ ]:
# Split the dataset into three separate dataframes based on the categorical column
df1 = df[df['Job Title'] == 0]
df2 = df[df['Job Title'] == 1]
df3 = df[df['Job Title'] == 2]


In [ ]:
#Model for Carer

X = df1.iloc[:, :-1]
y = df1[df1.columns[38]]
    
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
    
selector = SelectKBest(k=10)
selector.fit(x_train, y_train)
X_train_selected = selector.transform(x_train)
X_test_selected = selector.transform(x_test)

    # create a K-fold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # define the parameter grid
param_grid = {'n_estimators': [2, 10, 20],
                  'max_depth': [3, 5, 12],
                  'max_features': ['sqrt', 'log2']
                  }

    # create the random forest classifier
clf = RandomForestClassifier(random_state=42)

    # create the randomized search object
rs = RandomizedSearchCV(clf, param_grid, cv=kf, n_iter=18)

    # fit the randomized search object to the data
rs.fit(X_train_selected, y_train)
best_model = rs.best_estimator_

    # use the best model to predict on test data
y_pred = best_model.predict(X_test_selected)

    # calculate the test accuracy
test_accuracy = accuracy_score(y_test, y_pred)
    
report = classification_report(y_test, y_pred)
    
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))
print("Classification report:")
print(report)


/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Test accuracy: 89.99%
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.40      0.54      1898
           1       0.60      0.55      0.58      1863
           2       0.68      0.58      0.63      1874
           3       0.69      0.74      0.71      1835
           4       0.62      0.87      0.72      1869
           5       0.85      0.95      0.89      1851
           6       0.90      1.00      0.95      1888
           7       0.98      1.00      0.99      1962
           8       0.97      1.00      0.99      1953
           9       1.00      1.00      1.00      1857
          10       1.00      1.00      1.00      1832
          11       0.99      1.00      1.00      1866
          12       1.00      1.00      1.00      1923
          14       1.00      1.00      1.00      1859
          17       1.00      1.00      1.00      1824
          18       1.00      1.00      1.00      1886
          19       1.00      1.00   

In [ ]:
# Model for senior carer

X = df2.iloc[:, :-1]
y = df2[df2.columns[38]]
    
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
    
selector = SelectKBest(k=10)
selector.fit(x_train, y_train)
X_train_selected = selector.transform(x_train)
X_test_selected = selector.transform(x_test)

    # create a K-fold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # define the parameter grid
param_grid = {'n_estimators': [2, 3, 5],
              'max_depth': [3, 5, 8],
              'max_features': [ 'sqrt', 'log2']
              }

    # create the random forest classifier
clf = RandomForestClassifier(random_state=42)

    # create the randomized search object
rs = RandomizedSearchCV(clf, param_grid, cv=kf, n_iter=18)

    # fit the randomized search object to the data
rs.fit(X_train_selected, y_train)
best_model = rs.best_estimator_

    # use the best model to predict on test data
y_pred = best_model.predict(X_test_selected)

    # calculate the test accuracy
test_accuracy = accuracy_score(y_test, y_pred)
    
report = classification_report(y_test, y_pred)
    
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))
print("Classification report:")
print(report)


/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 2 12] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Test accuracy: 87.80%
Classification report:
              precision    recall  f1-score   support

           0       0.84      0.47      0.60       136
           1       0.62      0.73      0.67       137
           2       0.75      0.55      0.64       155
           3       0.85      0.54      0.66       162
           4       0.59      0.89      0.71       142
           5       0.80      1.00      0.89       165
           6       1.00      1.00      1.00       145
           7       0.91      1.00      0.96       150
           8       0.96      1.00      0.98       146
          10       0.99      1.00      1.00       134
          11       1.00      1.00      1.00       149
          12       1.00      1.00      1.00       150
          15       1.00      1.00      1.00       137
          19       1.00      1.00      1.00       150
          21       1.00      1.00      1.00       138

    accuracy                           0.88      2196
   macro avg       0.89      0.88  

In [ ]:
#Model for nursing

X = df3.iloc[:, :-1]
y = df3[df3.columns[38]]
    
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
    
selector = SelectKBest(k=10)
selector.fit(x_train, y_train)
X_train_selected = selector.transform(x_train)
X_test_selected = selector.transform(x_test)

    # create a K-fold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # define the parameter grid
param_grid = {'n_estimators': [2, 3, 5],
              'max_depth': [3, 5, 10],
              'max_features': [ 'sqrt', 'log2']
              }

    # create the random forest classifier
clf = RandomForestClassifier(random_state=42)

    # create the randomized search object
rs = RandomizedSearchCV(clf, param_grid, cv=kf, n_iter=18)

    # fit the randomized search object to the data
rs.fit(X_train_selected, y_train)
best_model = rs.best_estimator_

    # use the best model to predict on test data
y_pred = best_model.predict(X_test_selected)

    # calculate the test accuracy
test_accuracy = accuracy_score(y_test, y_pred)
    
report = classification_report(y_test, y_pred)
    
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))
print("Classification report:")
print(report)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Test accuracy: 89.25%
Classification report:
              precision    recall  f1-score   support

           0       0.41      0.61      0.49       238
           1       0.65      0.46      0.54       237
           2       0.64      0.54      0.59       246
           3       0.81      0.63      0.71       274
           4       0.87      0.80      0.83       248
           5       0.93      1.00      0.96       254
           6       0.91      1.00      0.95       227
           7       1.00      1.00      1.00       251
           8       0.95      1.00      0.98       255
           9       1.00      1.00      1.00       259
          10       1.00      1.00      1.00       262
          12       1.00      1.00      1.00       252
          13       1.00      1.00      1.00       267
          15       0.98      1.00      0.99       223
          16       0.95      1.00      0.97       242
          17       1.00      1.00      1.00       260
          19       1.00      1.00   